In [71]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import keras
import numpy as np


In [72]:
#(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
import tensorflow_datasets as tfds
mnist = tfds.load(name='mnist')

In [73]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

In [74]:
# Discriminator Net
X = tf.placeholder(tf.float32, shape=[None, 784], name='X')

D_W1 = tf.Variable(xavier_init([784, 128]), name='D_W1')
D_b1 = tf.Variable(tf.zeros(shape=[128]), name='D_b1')

D_W2 = tf.Variable(xavier_init([128, 1]), name='D_W2')
D_b2 = tf.Variable(tf.zeros(shape=[1]), name='D_b2')

theta_D = [D_W1, D_W2, D_b1, D_b2]



In [75]:

def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit

In [76]:
# Generator Net
Z = tf.placeholder(tf.float32, shape=[None, 100], name='Z')

G_W1 = tf.Variable(xavier_init([100, 128]), name='G_W1')
G_b1 = tf.Variable(tf.zeros(shape=[128]), name='G_b1')

G_W2 = tf.Variable(xavier_init([128, 784]), name='G_W2')
G_b2 = tf.Variable(tf.zeros(shape=[784]), name='G_b2')

theta_G = [G_W1, G_W2, G_b1, G_b2]


In [77]:

def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob

In [29]:
G_sample = generator(Z)
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
G_loss = -tf.reduce_mean(tf.log(D_fake))

In [78]:
def next_batch(current_pos, batch_size, end_pos, size, train_data, train_labels):
        """Either gets the next batch, or optionally shuffles and starts a
        new epoch."""
        end_pos = current_pos + batch_size
        if end_pos < size:
            batch = (train_data[current_pos:end_pos],
                     train_labels[current_pos:end_pos])
            current_pos +=batch_size
        else:
            # we return what's left (-> possibly smaller batch!) and prepare
            # the start of a new epoch
            batch = (train_data[current_pos:size],
                     train_labels[current_pos:size])
            current_pos = 0
            print("Starting new epoch...")
        return batch

In [83]:
# Only update D(X)'s parameters, so var_list = theta_D
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
# Only update G(X)'s parameters, so var_list = theta_G
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

mb_size = 128
Z_dim = 100

sess = tf.Session()
sess.run(tf.global_variables_initializer())

def sample_Z(m, n):
    '''Uniform prior for G(Z)'''
    return np.random.uniform(-1., 1., size=[m, n])

i = 0
train = mnist["train"]

test = mnist["test"]
for it in range(1000000):
    print("AAAAAA")
    print(train[0])
    #X_mb = next_batch(i, mb_size, )
    #i = i+128
    

    #_, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    #_, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(mb_size, Z_dim)})

ValueError: No gradients provided for any variable, check your graph for ops that do not support gradients, between variables ["<tf.Variable 'D_W1_2:0' shape=(784, 128) dtype=float32_ref>", "<tf.Variable 'D_W2_2:0' shape=(128, 1) dtype=float32_ref>", "<tf.Variable 'D_b1_2:0' shape=(128,) dtype=float32_ref>", "<tf.Variable 'D_b2_2:0' shape=(1,) dtype=float32_ref>"] and loss Tensor("Neg_2:0", shape=(), dtype=float32).